In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
import pandas as pd 
import numpy as np

Importing dataset 

In [3]:
df1=pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
df1.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
df1.shape

(7613, 5)

In [5]:
df1.isna().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [6]:
df1.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

# Case1: Drop all the null in the data set.
(This also makes the dataset al little more balanced)

In [7]:
df1.dropna(inplace=True,axis=0)
df1.shape

(5080, 5)

In [8]:
df1.target.value_counts()


0    2884
1    2196
Name: target, dtype: int64

**Pre-process the data:**

In [9]:
import spacy
nlp=spacy.load("en_core_web_lg")
def preprocess(text):
    doc=nlp(text.lower())
    filtered_data=[]
    for token in doc:
        if token.is_stop or token.is_punct or token.like_url:
            continue
        filtered_data.append(token.lemma_)
    return " ".join(filtered_data)

df1['prep_text']=df1['text'].apply(preprocess)
df1.head()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

,id,keyword,location,text,target,prep_text
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,@bbcmtd wholesale market ablaze
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,try bring heavy metal rt
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,africanbaze break news nigeria flag set ablaze...
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0,cry set ablaze
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,plus look sky night ablaze


For now we will add all the 3 columns i.e. keywords,location and text in one column namely new_text.

In [10]:
df1['new_text']=df1['keyword']+" "+df1['location']+" "+df1['prep_text']
df1.head()

,id,keyword,location,text,target,prep_text,new_text
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,@bbcmtd wholesale market ablaze,ablaze Birmingham @bbcmtd wholesale market ablaze
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,try bring heavy metal rt,ablaze Est. September 2012 - Bristol try bring...
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,africanbaze break news nigeria flag set ablaze...,ablaze AFRICA africanbaze break news nigeria f...
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0,cry set ablaze,"ablaze Philadelphia, PA cry set ablaze"
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,plus look sky night ablaze,"ablaze London, UK plus look sky night ablaze"


In [11]:
df1['new_text'][31]

'ablaze Birmingham @bbcmtd wholesale market ablaze'

**Split the data :**

In [12]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(
    df1['new_text'],df1.target,
    test_size=0.2,
    random_state=2022,
    stratify=df1.target
)

In [13]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((4064,), (1016,), (4064,), (1016,))

**Case 1.1: Apply Bag of Words model on various Classification models**

In [14]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
m1=Pipeline([
    ('BOW',CountVectorizer()),
    ('NB',MultinomialNB())    
])
m1.fit(x_train,y_train)
y_pred1=m1.predict(x_test)
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.81      0.80      0.81       577
           1       0.74      0.75      0.75       439

    accuracy                           0.78      1016
   macro avg       0.78      0.78      0.78      1016
weighted avg       0.78      0.78      0.78      1016



In [15]:
from sklearn.ensemble import RandomForestClassifier
m2=Pipeline([
    ('BOW',CountVectorizer()),
    ('RandomForest',RandomForestClassifier())    
])
m2.fit(x_train,y_train)
y_pred2=m2.predict(x_test)
print(classification_report(y_test,y_pred2))

              precision    recall  f1-score   support

           0       0.79      0.93      0.85       577
           1       0.88      0.67      0.76       439

    accuracy                           0.82      1016
   macro avg       0.83      0.80      0.80      1016
weighted avg       0.83      0.82      0.81      1016



In [16]:
from xgboost import XGBClassifier
m3=Pipeline([
    ('BOW',CountVectorizer()),
    ('XGB',XGBClassifier())    
])
m3.fit(x_train,y_train)
y_pred3=m3.predict(x_test)
print(classification_report(y_test,y_pred3))

              precision    recall  f1-score   support

           0       0.76      0.90      0.83       577
           1       0.83      0.63      0.72       439

    accuracy                           0.79      1016
   macro avg       0.80      0.77      0.77      1016
weighted avg       0.79      0.79      0.78      1016



In [17]:
from sklearn.linear_model import LogisticRegression
ml=Pipeline([
    ('BOW',CountVectorizer()),
    ('Lreg',LogisticRegression())    
])
ml.fit(x_train,y_train)
y_predl=ml.predict(x_test)
print(classification_report(y_test,y_predl))

              precision    recall  f1-score   support

           0       0.81      0.87      0.84       577
           1       0.81      0.73      0.77       439

    accuracy                           0.81      1016
   macro avg       0.81      0.80      0.80      1016
weighted avg       0.81      0.81      0.81      1016



**Case 1.2: Apply Bag of n-grams model on various Classification models**

In [18]:
m4=Pipeline([
    ('Ngram',CountVectorizer(ngram_range=(1,3))),
    ('NB',MultinomialNB())    
])
m4.fit(x_train,y_train)
y_pred4=m4.predict(x_test)
print(classification_report(y_test,y_pred4))

              precision    recall  f1-score   support

           0       0.81      0.84      0.82       577
           1       0.78      0.73      0.75       439

    accuracy                           0.79      1016
   macro avg       0.79      0.79      0.79      1016
weighted avg       0.79      0.79      0.79      1016



In [19]:
m5=Pipeline([
    ('Ngrams',CountVectorizer(ngram_range=(1,3))),
    ('RandomForest',RandomForestClassifier())    
])
m5.fit(x_train,y_train)
y_pred5=m5.predict(x_test)
print(classification_report(y_test,y_pred5))

              precision    recall  f1-score   support

           0       0.74      0.95      0.83       577
           1       0.90      0.55      0.68       439

    accuracy                           0.78      1016
   macro avg       0.82      0.75      0.76      1016
weighted avg       0.81      0.78      0.77      1016



In [20]:
from xgboost import XGBClassifier
m6=Pipeline([
    ('NG',CountVectorizer(ngram_range=(1,3))),
    ('XGB',XGBClassifier())    
])
m6.fit(x_train,y_train)
y_pred6=m6.predict(x_test)
print(classification_report(y_test,y_pred6))

              precision    recall  f1-score   support

           0       0.76      0.91      0.83       577
           1       0.84      0.63      0.72       439

    accuracy                           0.79      1016
   macro avg       0.80      0.77      0.78      1016
weighted avg       0.80      0.79      0.78      1016



In [21]:
from sklearn.linear_model import LogisticRegression
ml=Pipeline([
    ('BOW',CountVectorizer(ngram_range=(1,2))),
    ('Lreg',LogisticRegression())    
])
ml.fit(x_train,y_train)
y_predl=ml.predict(x_test)
print(classification_report(y_test,y_predl))

              precision    recall  f1-score   support

           0       0.80      0.88      0.84       577
           1       0.82      0.71      0.76       439

    accuracy                           0.81      1016
   macro avg       0.81      0.80      0.80      1016
weighted avg       0.81      0.81      0.80      1016



**Case 1.3: Apply TFIDF on various Classification models**

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
m7=Pipeline([
    ('tfidf',TfidfVectorizer()),
    ('NB',MultinomialNB())    
])
m7.fit(x_train,y_train)
y_pred7=m7.predict(x_test)
print(classification_report(y_test,y_pred7))

              precision    recall  f1-score   support

           0       0.78      0.87      0.82       577
           1       0.80      0.68      0.74       439

    accuracy                           0.79      1016
   macro avg       0.79      0.78      0.78      1016
weighted avg       0.79      0.79      0.79      1016



In [23]:
m8=Pipeline([
    ('tfidf1',TfidfVectorizer()),
    ('RandomForest',RandomForestClassifier())    
])
m8.fit(x_train,y_train)
y_pred8=m8.predict(x_test)
print(classification_report(y_test,y_pred8))

              precision    recall  f1-score   support

           0       0.78      0.87      0.82       577
           1       0.80      0.67      0.73       439

    accuracy                           0.79      1016
   macro avg       0.79      0.77      0.78      1016
weighted avg       0.79      0.79      0.78      1016



In [24]:
m9=Pipeline([
    ('tfidf2',TfidfVectorizer()),
    ('XGB',XGBClassifier())    
])
m9.fit(x_train,y_train)
y_pred9=m9.predict(x_test)
print(classification_report(y_test,y_pred9))

              precision    recall  f1-score   support

           0       0.76      0.91      0.83       577
           1       0.84      0.62      0.71       439

    accuracy                           0.78      1016
   macro avg       0.80      0.76      0.77      1016
weighted avg       0.79      0.78      0.78      1016



In [25]:
from sklearn.linear_model import LogisticRegression
ml=Pipeline([
    ('BOW',TfidfVectorizer()),
    ('L_reg',LogisticRegression())    
])
ml.fit(x_train,y_train)
y_predl=ml.predict(x_test)
print(classification_report(y_test,y_predl))

              precision    recall  f1-score   support

           0       0.79      0.89      0.84       577
           1       0.82      0.69      0.75       439

    accuracy                           0.80      1016
   macro avg       0.81      0.79      0.80      1016
weighted avg       0.81      0.80      0.80      1016



Applying the "ml" model consisting of the Boag of Words and the logistic regression model for the test dataset as it performed the best with an accuracy of 81% and weighed avg of 81%

In [26]:
df2=pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
df2.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [27]:
df2.shape

(3263, 4)

In [28]:
df2.isna().sum()

id             0
keyword       26
location    1105
text           0
dtype: int64

In [29]:
df2.dropna(inplace=True,axis=0)
df2.shape

(2158, 4)

In [30]:
df2['prep_text']=df2['text'].apply(preprocess)
df2.head()

,id,keyword,location,text,prep_text
15,46,ablaze,London,Birmingham Wholesale Market is ablaze BBC News...,birmingham wholesale market ablaze bbc news fi...
16,47,ablaze,Niall's place | SAF 12 SQUAD |,@sunkxssedharry will you wear shorts for race ...,@sunkxssedharry wear short race ablaze
17,51,ablaze,NIGERIA,#PreviouslyOnDoyinTv: Toke MakinwaÛªs marriag...,previouslyondoyintv toke makinwaûªs marriage ...
18,58,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http:/...,check nsfw
19,60,ablaze,"Los Angeles, Califnordia",PSA: IÛªm splitting my personalities.\n\n?? t...,psa iûªm split personality \n\n techie follow...


In [31]:
df2['n_text']=df2['keyword']+" "+df2['location']+" "+df2['prep_text']
df2.head()

,id,keyword,location,text,prep_text,n_text
15,46,ablaze,London,Birmingham Wholesale Market is ablaze BBC News...,birmingham wholesale market ablaze bbc news fi...,ablaze London birmingham wholesale market abla...
16,47,ablaze,Niall's place | SAF 12 SQUAD |,@sunkxssedharry will you wear shorts for race ...,@sunkxssedharry wear short race ablaze,ablaze Niall's place | SAF 12 SQUAD | @sunkxss...
17,51,ablaze,NIGERIA,#PreviouslyOnDoyinTv: Toke MakinwaÛªs marriag...,previouslyondoyintv toke makinwaûªs marriage ...,ablaze NIGERIA previouslyondoyintv toke makinw...
18,58,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http:/...,check nsfw,ablaze Live On Webcam check nsfw
19,60,ablaze,"Los Angeles, Califnordia",PSA: IÛªm splitting my personalities.\n\n?? t...,psa iûªm split personality \n\n techie follow...,"ablaze Los Angeles, Califnordia psa iûªm spli..."


In [32]:
df2['n_text'][19]

'ablaze Los Angeles, Califnordia psa i\x89ûªm split personality \n\n techie follow @ablaze_co \n burner follow @ablaze'

In [33]:
y_pred_test1=ml.predict(df2.n_text)
result=pd.DataFrame(y_pred_test1,columns=['target'])
result.head()

,target
0,1
1,0
2,1
3,0
4,0


In [34]:
result.value_counts()

target
0         1397
1          761
dtype: int64